In [2]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [15]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# for pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user sound and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

In [17]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url,payload)

In [18]:
push("Hey !")

Push: Hey !


In [19]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded":"ok"}

In [20]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded":"ok"}

In [21]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "the email address of this user"
            },
            "name":{
                "type": "string",
                "description": "the user's name, if they provided it"
            },
            "notes":{
                "type": "string",
                "description": "Any additional infomration about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [22]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "the question that couldn't be answered"
            },
            "required": ["question"],
            "additionalProperties": False
        }
    }
}

In [23]:
tools = [{"type": "function","function": record_unknown_question_json},
         {"type": "function","function": record_user_details_json}]

In [ ]:
tools

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}, flush=True")

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result == record_unknown_question(**arguments)

        results.append({"role": "tool", "content": json.dumps(result),"tool_call_id":tool_call.id})
    return results